In [1]:
from src.adapter.chromadb import ChromaDB
from src.service.text_embedder.sentence_transformer import TextEmbedding
from src.service.text_generation.local_transformers import LocalTransformerTextGeneration
from src.service.classification.local_transformers import LocalTransformerClassification
from src.service.reranking.local_transformers import LocalTransformerReranking
from src.service.document_retrieval.chromadb import ChromaDocumentRetrievalService

text_embedding_service = TextEmbedding(model_id="BAAI/bge-m3")
chromadb = ChromaDB(embedding_service=text_embedding_service)
text_generation_service = LocalTransformerTextGeneration(
    model_id="meta-llama/Meta-Llama-3-8B-Instruct",
    device="cuda",
    attn_implementation="sdpa"
)
classification_service = LocalTransformerClassification(
    model_id="MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
)
reranking_service = LocalTransformerReranking(
    model_id='BAAI/bge-reranker-v2-m3'
)
document_retrieval_service = ChromaDocumentRetrievalService(
    client=chromadb.get_client(),
    embedding_service=chromadb.get_embedding_service()
)

/home/buddy/Study-Buddy/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/buddy/Study-Buddy/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.47it/s]


Loaded Text Generator model: meta-llama/Meta-Llama-3-8B-Instruct
Loaded Cross-Encoder model: BAAI/bge-reranker-v2-m3


In [2]:
import importlib
from src import rag_system, utils, task_manager

# Reload the modules
importlib.reload(rag_system)
importlib.reload(utils.config_manager)
importlib.reload(task_manager)

from src.rag_system import RAGSystem
from src.utils.config_manager import ConfigManager
from src.task_manager import TaskManager

config = ConfigManager(env='dev')

rag_system = RAGSystem(
  config=config, 
  text_generation_service=text_generation_service,
  classification_service=classification_service,
  reranking_service=reranking_service,
  document_retrieval_service=document_retrieval_service
)

GLOBAL ConfigManager init...
. . . Loading config from /home/buddy/Study-Buddy/config/dev.yaml



In [8]:
dataset = [
  {
    "query": "What is the name of the muscular tube that transports food from the mouth to the stomach, and does any chemical digestion occur there?",
    "passages": "ns_21",
    "explanation": """This question requires specific information about the esophagus and its role in digestion. Passage 21 directly addresses both parts of the question.""",
    "ground_truth": "The muscular tube that transports food from the mouth to the stomach is called the **esophagus**. No chemical digestion occurs in the esophagus.",
  },
  {
    "query": "What are the two main types of starch, and how do they differ in their structure?",
    "passages": "ns_56",
    "explanation": """This question requires understanding the two types of starch and their chemical structures. Passage 56 defines amylose and amylopectin and explains how their structures differ.""",
    "ground_truth": "The two main types of starch are **amylose** and **amylopectin**. **Amylose** is made up of straight chains of glucose molecules, while **amylopectin** is made up of branched chains of glucose molecules.",
  },
  {
    "query": "What is the name of the hormone secreted by adipose tissue that helps regulate appetite, and how does it work?",
    "passages": "ns_119, ns_232",
    "explanation": """This question focuses on the specific hormone leptin and requires understanding its role in signaling energy balance.  Passage 119 mentions leptin's role in regulating appetite, and passage 232 explains how it works and why it may not be an effective weight loss drug.""",
    "ground_truth": "The hormone secreted by adipose tissue that helps regulate appetite is called **leptin**. As fat stores increase, more leptin is produced. Leptin acts on the brain to suppress hunger and increase energy expenditure. However, people who are overweight or obese are often resistant to leptin, meaning that their brains don't respond to it as effectively.",
  },
  {
    "query": "Explain the three mechanisms by which fluoride combats tooth decay.",
    "passages": "ns_382",
    "explanation": """This question requires specific knowledge about the action of fluoride on teeth. Passage 382 directly addresses this by listing and explaining the three mechanisms.""",
    "ground_truth": "Fluoride combats tooth decay by: \n\n1. **Blocking acid formation by bacteria**\n2. **Preventing demineralization of teeth**\n3. **Enhancing remineralization of destroyed enamel**",
  },
  {
    "query": "What is the recommended daily intake of protein for athletes, and how does this compare to the RDA for sedentary adults?",
    "passages": "ns_193, ns_444",
    "explanation": """This question focuses on the specific protein recommendations for athletes and requires comparing them to the standard RDA. Passage 193 provides the RDA for sedentary adults, and passage 444 provides the recommendations for athletes.""",
    "ground_truth": "The recommended protein intake for athletes is **1.2 to 2.0 grams per kilogram of body weight per day**, depending on the type and intensity of training. This is higher than the RDA for sedentary adults, which is **0.8 grams per kilogram of body weight per day**.",
  },
  {
    "query": "What is the name of the enzyme produced in the mouth that begins the chemical breakdown of triglycerides?",
    "passages": "ns_20",
    "explanation": "Passage 20 details the beginning of lipid digestion in the mouth and names the specific enzyme involved.",
    "ground_truth": "The enzyme produced in the mouth that begins the chemical breakdown of triglycerides is called **lingual lipase**.",
  },
  {
    "query": "What is the difference between a monounsaturated fatty acid and a polyunsaturated fatty acid?",
    "passages": "ns_134",
    "explanation": "Passage 134 defines both monounsaturated and polyunsaturated fatty acids, focusing on their structural differences.",
    "ground_truth": "A **monounsaturated fatty acid** has **one double bond** between carbon atoms in its chain, while a **polyunsaturated fatty acid** has **two or more double bonds**.",
  },
  {
    "query": "Explain the process of emulsification and why it is important for lipid digestion.",
    "passages": "ns_146",
    "explanation": "Passage 146 explains the role of bile in emulsifying fat and why this is necessary for lipid digestion in the small intestine.",
    "ground_truth": "Emulsification is the process of breaking down large fat globules into smaller droplets. This is important for lipid digestion because it increases the surface area of the fat, allowing digestive enzymes called lipases to more effectively break down the triglycerides. Bile, produced in the liver and stored in the gallbladder, is released into the small intestine and acts as an emulsifier.",
  },
  {
    "query": "What are the names of the two hormones involved in blood calcium regulation, and which gland produces each?",
    "passages": "ns_369",
    "explanation": "Passage 369 describes the process of blood calcium regulation, naming the two key hormones and identifying the glands that produce them.",
    "ground_truth": "The two hormones involved in blood calcium regulation are: \n\n1. **Parathyroid hormone (PTH)**, produced by the **parathyroid glands**. PTH increases blood calcium levels.\n2. **Calcitonin**, produced by the **thyroid gland**. Calcitonin decreases blood calcium levels.",
  },
  {
    "query": "Why is it important to introduce potentially allergenic foods to babies during their first year of life?",
    "passages": "ns_477, ns_478",
    "explanation": "Passages 477 and 478 address the rationale for introducing allergenic foods in infancy, including the evidence that this practice can prevent the development of food allergies.",
    "ground_truth": "Introducing potentially allergenic foods to babies during their first year of life can help prevent food allergies from developing. Studies show that early introduction of foods like peanut and egg can significantly reduce the risk of developing allergies to those foods.",
  }
]

In [4]:
from src.models.context import Context
results = []

for instance in dataset:
  query = instance['query']
  ground_truth = instance.get('ground_truth', None)
  response = await rag_system.process_query(query, eval = True)
  results.append(response)
  #results.append({
  #  'query': query,
  #  'response': response.answer,
  #  'ground_truth': ground_truth
  #})

Route: question
Query Doc Pairs:
10
QUERY: what is the name of the muscular tube that transports food from the mouth to the stomach and does any chemical digestion occur there
DOCUMENT: ns_21
The esophagus is a muscular tube that transports food from the mouth to the stomach. No chemical digestion occurs while the bolus is mechanically propelled through this tube by peristalsis.
GRADE: yes
DOCUMENT: ns_18
The process of digestion includes five main activities: ingestion, mechanical digestion, chemical digestion, absorption, and excretion.
The first of these processes, ingestion, refers to the entry of food into the GI tract through the mouth. There, the food is chewed and mixed with saliva, which contains enzymes that begin breaking down the carbohydrates and lipids in food. Mastication (chewing) increases the surface area of the food and allows for food to be broken into small enough pieces to be swallowed safely.
Food (now called a bolus since it has been chewed and moistened) leaves

In [31]:
results[2].retrieved_documents.documents
results[2].response.text

"The hormone secreted by adipose tissue that helps regulate appetite is leptin [ns_119]. Leptin plays a crucial role in signaling the body's energy status, and its mechanism of action is intricately linked to its production and reception in the brain.\n\nWhen fat stores increase, leptin is produced in response, serving as a signal to the brain that the body is in a state of positive energy balance [ns_232]. This hormone then communicates with the satiety center in the hypothalamus, conveying the message that the body's energy needs are being met. As a result, leptin acts on the brain to suppress hunger and increase energy expenditure, thereby regulating appetite and maintaining a healthy energy balance."

In [34]:
query_passages= []
for instance, result in zip(dataset, results):
  query = instance['query']
  ground_truth = instance.get('ground_truth', None)
  rag_response = result.response.text
  ground_truth_passages = []
  rag_response_passages = []
  for doc in result.retrieved_documents.documents:
    if doc.id in instance['passages']:
      ground_truth_passages.append({
        "id": doc.id,
        "passage": doc.document
        })
    else:
      rag_response_passages.append({
        "id": doc.id,
        "passage": doc.document
      })
  query_passages.append({
    "query": query,
    "ground_truth": ground_truth,
    "ground_truth_passages": ground_truth_passages,
    "rag_response": rag_response,
    "other_passages": rag_response_passages
  })

  """
  {
    "query": "The original query",
    "ground_truth": "The ground truth answer",
    "ground_truth_passages": [
      {
        "id": "ns_21", "passage": "Passed that was used to answer the query and generate the ground truth"
      },
      {
        "id": "ns_56", "passage": "Another passage that was used to answer the query and generate the ground truth"
      },
      ...
    ],
    "rag_response": "The response generated by the RAG system",
    "other_passages": [
      {
        "id": "ns_119", "passage": "Another passage that was retrieved by the RAG system that MIGHT have been used to generate the RAG response"
      },
      {
        "id": "ns_232", "passage": "Another passage that was retrieved by the RAG system that MIGHT have been used to generate the RAG response"
      },
      ...
  }
  """

query_passages
  #results.append({
  #  'query': query,
  #  'response': response.answer,
  #  'ground_truth': ground_truth
  #})

[{'query': 'What is the name of the muscular tube that transports food from the mouth to the stomach, and does any chemical digestion occur there?',
  'ground_truth': 'The muscular tube that transports food from the mouth to the stomach is called the **esophagus**. No chemical digestion occurs in the esophagus.',
  'ground_truth_passages': [{'id': 'ns_21',
    'passage': 'The esophagus\xa0is a muscular tube that transports food from the mouth to the stomach. No chemical digestion occurs while the bolus is mechanically propelled through this tube by peristalsis.'}],
  'rag_response': 'The muscular tube that transports food from the mouth to the stomach is the esophagus [ns_21]. As for chemical digestion, it does not occur in the esophagus [ns_21, ns_18].',
  'other_passages': [{'id': 'ns_18',
    'passage': 'The process of digestion includes five main activities: ingestion, mechanical digestion, chemical digestion, absorption, and excretion.\nThe first of these processes, ingestion, ref

In [6]:
import pandas as pd


extracted_data = []

for result in results:
  query_text = result.query.text
  for doc in result.retrieved_documents.documents:
    extracted_data.append({
      "query": query_text,
      "document": f"[{doc.id}] {doc.document}",
      "transformed_document": doc.transformed_document,
    })

df_summaries = pd.DataFrame(extracted_data)

extracted_data = []
for i, result in enumerate(results):
  query_text = result.query.text

  extracted_data.append({
    "query": query_text,
    "response": result.response.text,
    #"ground_truth": dataset[i].get('ground_truth', None)
  })

df_responses = pd.DataFrame(extracted_data)

df_responses.to_json('df_responses.json', orient='records', lines=True, indent=2, force_ascii=False)
df_summaries.to_json('df_summaries.json', orient='records', lines=True, indent=2)


In [12]:
df_summaries

,query,document,transformed_document
0,how do bile salts contribute to both the emuls...,"[ns_147] Next, those products of fat digestion...",Bile salts cluster around products of fat dige...
1,how do bile salts contribute to both the emuls...,[ns_146] As the stomach contents enter the sma...,"Bile salts, with their hydrophobic and hydroph..."
2,how do bile salts contribute to both the emuls...,[ns_23] Chyme released from the stomach enters...,"Bile salts, an emulsifier, break down large fa..."
3,how do bile salts contribute to both the emuls...,[ns_143] Lipid digestion and absorption pose s...,Bile salts emulsify lipids in the small intest...
4,explain how a deficiency in vitamin d can nega...,[ns_389] Vitamin D synthesis in the skin begin...,A deficiency in vitamin D can negatively affec...
5,explain how a deficiency in vitamin d can nega...,"[ns_392] In children, vitamin D deficiency cau...",A deficiency in vitamin D can negatively affec...
6,explain how a deficiency in vitamin d can nega...,[ns_364] The best activities for stimulating n...,A deficiency in vitamin D is a strong risk fac...
7,explain how a deficiency in vitamin d can nega...,[ns_363] Osteoporosis is a bone disease that o...,A deficiency in vitamin D can negatively affec...
8,explain how a deficiency in vitamin d can nega...,"[ns_390] In most people, vitamin D synthesis i...",A deficiency in vitamin D can negatively affec...
9,how does the process of photosynthesis contrib...,[ns_9] Photosynthesis is essential to all life...,Photosynthesis is essential for the food we ea...


In [20]:
import json

output = []

for result, instance in zip(results, dataset):
    output.append({
        "query": result.query.text,
        "response": result.response.text,
        "ground_truth": instance['ground_truth'],
        "retrieved_documents": [doc.id + " " + doc.transformed_document for doc in result.retrieved_documents.documents]
    })

json_output = json.dumps(output, indent=2)
print(json_output)

with open('eval1.json', 'w') as json_file:
    json_file.write(json_output)

[
  {
    "query": "what is the name of the muscular tube that transports food from the mouth to the stomach and does any chemical digestion occur there",
    "response": "The muscular tube that transports food from the mouth to the stomach is the esophagus [ns_21]. As for chemical digestion, it does not occur in the esophagus [ns_21, ns_18].",
    "ground_truth": "The muscular tube that transports food from the mouth to the stomach is called the **esophagus**. No chemical digestion occurs in the esophagus.",
    "retrieved_documents": [
      "ns_21 The muscular tube that transports food from the mouth to the stomach is the esophagus. No chemical digestion occurs in the esophagus.",
      "ns_18 The muscular tube that transports food from the mouth to the stomach is the esophagus. Chemical digestion does not occur in the esophagus, but rather in the small intestine, where digestive secretions containing enzymes break down macronutrients into their chemical building blocks.",
      "ns

In [11]:
import json

queries = []
responses = []
ground_truths = []

# Populate lists
for result, instance in zip(results, dataset):
    queries.append(result.query.text)
    responses.append(result.response.text)
    ground_truths.append(instance['ground_truth'])

output = {
    "queries": queries,
    "responses": responses,
    "ground_truths": ground_truths
}

# Convert dictionary to JSON string with indentation
json_output = json.dumps(output, indent=2)

# Print JSON output
print(json_output)

# Write JSON output to a file
with open('eval_list.json', 'w') as json_file:
    json_file.write(json_output)

{
  "queries": [
    "what is the name of the muscular tube that transports food from the mouth to the stomach and does any chemical digestion occur there",
    "what are the two main types of starch and how do they differ in their structure",
    "what is the name of the hormone secreted by adipose tissue that helps regulate appetite and how does it work",
    "explain the three mechanisms by which fluoride combats tooth decay",
    "what is the recommended daily intake of protein for athletes and how does this compare to the rda for sedentary adults",
    "what is the name of the enzyme produced in the mouth that begins the chemical breakdown of triglycerides",
    "what is the difference between a monounsaturated fatty acid and a polyunsaturated fatty acid",
    "explain the process of emulsification and why it is important for lipid digestion",
    "what are the names of the two hormones involved in blood calcium regulation and which gland produces each",
    "why is it important t

In [17]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
from bert_score import score
from rouge import Rouge
import nltk
nltk.download('wordnet')

rouge = Rouge()

rouge_l_scores = []
bleu_scores = []
meteor_scores = []


# Example data (replace with your actual data)
P, R, F1 = score(responses, ground_truths, lang="en", verbose=True)

print(f"Average BERTScore F1: {F1.mean()}")

for query, ground_truth, response in zip(queries, ground_truths, responses):
    rouge_scores = rouge.get_scores(response, ground_truth)
    rouge_l_scores.append(rouge_scores[0]['rouge-l']['f'])

    bleu_scores.append(sentence_bleu([ground_truth.split()], response.split()))

    meteor_scores.append(single_meteor_score(ground_truth.split(), response.split()))

    
for rouge_l, bleu, meteor in zip(rouge_l_scores, bleu_scores, meteor_scores):
    print(f"ROUGE-L: {rouge_l}, BLEU: {bleu}, METEOR: {meteor}")


avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_meteor = sum(meteor_scores) / len(meteor_scores)

print(f"Average ROUGE-L: {avg_rouge_l}")
print(f"Average BLEU: {avg_bleu}")
print(f"Average METEOR: {avg_meteor}")

ROUGE-L: 0.6666666617283951, BLEU: 0.45443623637127284, METEOR: 0.7585268884703913
ROUGE-L: 0.25287355955608404, BLEU: 0.028419549417753945, METEOR: 0.2147971360381862
ROUGE-L: 0.4516128984131114, BLEU: 0.1870441036625162, METEOR: 0.4820615931101504
ROUGE-L: 0.15942028727473223, BLEU: 8.348472855689752e-79, METEOR: 0.17361111111111113
ROUGE-L: 0.4952380907936509, BLEU: 0.10711388838062348, METEOR: 0.5319647946833956
ROUGE-L: 0.6046511581179016, BLEU: 0.37184214350816, METEOR: 0.645760963960052
ROUGE-L: 0.14285714050177095, BLEU: 6.076398446407336e-79, METEOR: 0.1449275362318841
ROUGE-L: 0.33540372252305084, BLEU: 0.04903382977862074, METEOR: 0.3859640831758034
ROUGE-L: 0.2752293540712062, BLEU: 0.05259899630112923, METEOR: 0.41902314622529824
ROUGE-L: 0.35443037644207664, BLEU: 0.09686207813569611, METEOR: 0.45767062960838056
Average ROUGE-L: 0.373838324942198
Average BLEU: 0.13473508255557726
Average METEOR: 0.4214307882614653


[nltk_data] Downloading package wordnet to /home/buddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/buddy/Study-Buddy/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [36]:
import json
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
from bert_score import score
from rouge import Rouge
import nltk
nltk.download('wordnet')

rouge = Rouge()

rouge_l_scores = []
bleu_scores = []
meteor_scores = []
bert_scores = []

# Example data (replace with your actual data)
P, R, F1 = score(responses, ground_truths, lang="en", verbose=True)

print(f"Average BERTScore F1: {F1.mean()}")

query_scores = []

for query, ground_truth, response, bert_f1 in zip(queries, ground_truths, responses, F1):
    rouge_scores = rouge.get_scores(response, ground_truth)
    rouge_l = rouge_scores[0]['rouge-l']['f']
    rouge_l_scores.append(rouge_l)

    bleu = sentence_bleu([ground_truth.split()], response.split())
    bleu_scores.append(bleu)

    meteor = single_meteor_score(ground_truth.split(), response.split())
    meteor_scores.append(meteor)

    bert_scores.append(bert_f1.item())

    query_scores.append({
        "query": query,
        "rouge_l": rouge_l,
        "bleu": bleu,
        "meteor": meteor,
        "bert_score": bert_f1.item()
    })

avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_meteor = sum(meteor_scores) / len(meteor_scores)
avg_bert = sum(bert_scores) / len(bert_scores)

print(f"Average ROUGE-L: {avg_rouge_l}")
print(f"Average BLEU: {avg_bleu}")
print(f"Average METEOR: {avg_meteor}")
print(f"Average BERTScore: {avg_bert}")

# Add average scores to the JSON output
query_scores.append({
    "query": "average",
    "rouge_l": avg_rouge_l,
    "bleu": avg_bleu,
    "meteor": avg_meteor,
    "bert_score": avg_bert
})

# Write the scores to a JSON file
with open('query_scores.json', 'w') as f:
    json.dump(query_scores, f, indent=4)

[nltk_data] Downloading package wordnet to /home/buddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  8.02it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 398.93it/s]

done in 0.13 seconds, 77.18 sentences/sec
Average BERTScore F1: 0.8843156695365906
Average ROUGE-L: 0.373838324942198
Average BLEU: 0.13473508255557726
Average METEOR: 0.4214307882614653
Average BERTScore: 0.8843156397342682


In [61]:
import json

# Load the JSON file
with open('llm_eval.json', 'r') as f:
    data = json.load(f)

# Extract keys from the different objects
keys = set()
for obj in data:
    keys.update(obj.keys())

# Print the keys
print("Keys in the JSON objects:", keys)
eval_auto_llm = []
for query, score, eval in zip(queries, query_scores, data):
  eval_auto_llm.append({
    "query": query,
    "automatic_evaluation": score,
    "llm_eval": eval
    })
    
with open('llm_eval_q.json', 'w') as f:
  json.dump(eval_auto_llm, f, indent=4)

Keys in the JSON objects: {'appropriateness_explanation', 'appropriateness_score', 'information_usage_score', 'relevance_score', 'accuracy_explanation', 'relevance_explanation', 'depth_and_elaboration_score', 'accuracy_score', 'improvement_suggestions', 'coherence_score', 'information_usage_explanation', 'coherence_explanation', 'essential_completeness_explanation', 'overall_score', 'overall_assessment', 'depth_explanation', 'essential_completeness_score'}


In [63]:
n_keys = set()

for obj in data:
  for key in obj.keys():
    if "_score" in key:
      
      n_keys.update([key.split('_score')[0] for key in obj.keys()])

n_keys

{'accuracy',
 'accuracy_explanation',
 'appropriateness',
 'appropriateness_explanation',
 'coherence',
 'coherence_explanation',
 'depth_and_elaboration',
 'depth_explanation',
 'essential_completeness',
 'essential_completeness_explanation',
 'improvement_suggestions',
 'information_usage',
 'information_usage_explanation',
 'overall',
 'overall_assessment',
 'relevance',
 'relevance_explanation'}

In [47]:
key_lists_text = {key: [] for key in keys if "_score" not in key}
key_lists_score = {key: [] for key in keys if "_score" in key}
# Populate the lists with values from the JSON objects
for obj in data:
    for key in keys:
      k = key.split("_")
      if k[1] == "score":
      if "_score" in key:
        key_lists_score[key].append(obj.get(key, None))
      else:
        key_lists_text[key].append(obj.get(key, None))

avg_scores = []
# Calculate the average scores
for key, values in key_lists_score.items():
    avg_score = sum(values) / len(values)
    avg_scores.append({
        "key": key,
        "average_score": avg_score
    })
    #key_lists_score[key] = avg_score

# Write the lists to a new JSON file
with open('key_lists_text.json', 'w') as f:
    json.dump(key_lists_text, f, indent=4)
with open('key_lists_score.json', 'w') as f:
    json.dump(key_lists_score, f, indent=4)
with open('avg_scores.json', 'w') as f:
    json.dump(avg_scores, f, indent=4)
# Print the keys
print("Keys in the JSON objects:", keys)

Keys in the JSON objects: {'appropriateness_explanation', 'appropriateness_score', 'information_usage_score', 'relevance_score', 'accuracy_explanation', 'relevance_explanation', 'depth_and_elaboration_score', 'accuracy_score', 'improvement_suggestions', 'coherence_score', 'information_usage_explanation', 'coherence_explanation', 'essential_completeness_explanation', 'overall_score', 'overall_assessment', 'depth_explanation', 'essential_completeness_score'}


In [7]:
import json




for i, result in enumerate(results):
  query_text = result.query.text
  documents = result.retrieved_documents.documents
  documents_list = [
    {
        'id': doc.id,
        'document': doc.document,
    }
    for doc in documents
  ]
  query_and_documents = {
    "query": query_text,
    "documents": documents_list
  }

  with open(f'query_{i}.json', 'w') as json_file:
    json.dump(query_and_documents, json_file, indent=4)
  

In [8]:
import json

for i, result in enumerate(results):
  query_text = result.query.text
  documents = result.retrieved_documents.documents
  documents_list = [
    {
        'id': doc.id,
        'summarized_document': doc.transformed_document,
    }
    for doc in documents
  ]
  query_and_documents = {
    "query": query_text,
    "summarized_documents": documents_list
  }

  with open(f'query_t_{i}.json', 'w') as json_file:
    json.dump(query_and_documents, json_file, indent=2)

In [5]:
import json

for i, result in enumerate(results):
  query_text = result.query.text
  response = result.response.text
  documents = result.retrieved_documents.documents
  documents_list = [
    {
        'id': doc.id,
        'summarized_document': doc.transformed_document,
    }
    for doc in documents
  ]
  query_and_documents = {
    "query": query_text,
    "summarized_documents": documents_list,
    "response": response
  }

  with open(f'query_t_{i}.json', 'w') as json_file:
    json.dump(query_and_documents, json_file, indent=2)

In [7]:
import json

for i, result in enumerate(results):
  query_text = result.query.text
  documents = result.retrieved_documents.documents
  documents_list = [
    {
        'id': doc.id,
        'full_document': doc.document,
        'summarized_document': doc.transformed_document,
    }
    for doc in documents
  ]
  query_and_documents = {
    "query": query_text,
    "documents": documents_list
  }

  with open(f'query__ft_{i}.json', 'w') as json_file:
    json.dump(query_and_documents, json_file, indent=2)

In [13]:

# Assuming result.retrieved_documents.documents is the list of DocumentObject instances



documents = results[0].retrieved_documents.documents


# Extract relevant attributes from each DocumentObject
documents_list = [
    {
        'id': doc.id,
        'document': doc.document,
    }
    for doc in documents
]

# Convert the list of dictionaries to a JSON string and save it to a file
with open('documents.json', 'w') as json_file:
    json.dump(documents_list, json_file, indent=4)

In [18]:
from src.task_manager import TaskManager

services = {
  "text_generation_service": text_generation_service,
  "classification_service": classification_service,
  "reranking_service": reranking_service,
  "document_retrieval_service": document_retrieval_service
}

task_manager = TaskManager(config._config, services)

In [20]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
from rouge import Rouge

rouge = Rouge()

# Example data (replace with your actual data)
queries = ["What is the name...", ...]
ground_truths = ["The muscular tube...", ...]
rag_responses = ["The muscular tube...", ...]

rouge_l_scores = []
bleu_scores = []
meteor_scores = []

for query, ground_truth, rag_response in zip(queries, ground_truths, rag_responses):
    rouge_scores = rouge.get_scores(rag_response, ground_truth)
    rouge_l_scores.append(rouge_scores[0]['rouge-l']['f'])

    bleu_scores.append(sentence_bleu([ground_truth.split()], rag_response.split()))

    meteor_scores.append(single_meteor_score(ground_truth, rag_response))

avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_meteor = sum(meteor_scores) / len(meteor_scores)

print(f"Average ROUGE-L: {avg_rouge_l}")
print(f"Average BLEU: {avg_bleu}")
print(f"Average METEOR: {avg_meteor}")

{'PreprocessQueryTask': <src.tasks.preprocess_query.PreprocessQueryTask object at 0x79d79039aec0>, 'ReformulateQueryTask': <src.tasks.reformulate_query.ReformulateQueryTask object at 0x79d7903987f0>, 'DecomposeQueryTask': <src.tasks.decompose_query.DecomposeQueryTask object at 0x79d79039a110>, 'ClassifyQueryTask': <src.tasks.classify_query.ClassifyQueryTask object at 0x79d79039af20>, 'ClassifyMultistepQueryTask': <src.tasks.classify_multistep_query.ClassifyMultistepQueryTask object at 0x79d790398ee0>, 'RetrieveDocumentsTask': <src.tasks.retrieve_documents.RetrieveDocumentsTask object at 0x79d790398e50>, 'FilterDocumentsTask': <src.tasks.filter_documents.FilterDocumentsTask object at 0x79d790398a60>, 'ReRankingTask': <src.tasks.rerank_documents.ReRankingTask object at 0x79d79039bd90>, 'GenerateResponseTask': <src.tasks.generate_response.GenerateResponseTask object at 0x79d79039abf0>, 'GradeResponseTask': <src.tasks.grade_response.GradeResponseTask object at 0x79d79039b880>, 'ComplexQuer

In [22]:
from src.models.context import Context
from src.models.query import Query

results = []
for instance in dataset:

  query = instance['query']
  context = Context(query=Query(text=query))
  
  context = await task_manager.execute_task('PreprocessQueryTask', context)
  print(context)
  context = await task_manager.execute_task('ClassifyQueryTask', context)
  print(context)



query=Query(text='how do bile salts contribute to both the emulsification and absorption of lipids in the small intestine', reformulated_query=None, query_history=[], decomposed_parts=[], label=None, is_multistep=False, embeddings=[]) retrieved_documents=RetrievedDocuments(documents=[], original_order=[], ignore=[], filtered_documents=[]) response=Response(text='', grade=None) last_task='PreprocessQueryTask' routing_key='default' response_type=None task_history=[]
Route: question
query=Query(text='how do bile salts contribute to both the emulsification and absorption of lipids in the small intestine', reformulated_query=None, query_history=[], decomposed_parts=[], label='question', is_multistep=False, embeddings=[]) retrieved_documents=RetrievedDocuments(documents=[], original_order=[], ignore=[], filtered_documents=[]) response=Response(text='', grade=None) last_task='PreprocessQueryTask' routing_key='question' response_type='question' task_history=[]
query=Query(text='explain how a d